In [30]:
import torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
sys.path.append('..')

In [3]:
from torchdyn.utils import plot_3D_dataset
from torchdyn.datasets import ToyDataset
from torch.utils.data import TensorDataset, DataLoader

In [39]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
dry_run = False

In [40]:
d = ToyDataset()
X, yn = d.generate(n_samples=512, dataset_type='moons', noise=.4)
X_train = torch.Tensor(X).to(device)
y_train = torch.LongTensor(yn.long()).to(device)
train = TensorDataset(X_train, y_train)
trainloader = DataLoader(train, batch_size=len(X), shuffle=False)

In [41]:
import torch.nn as nn
import pytorch_lightning as pl

class Learner(pl.LightningModule):
    def __init__(self, t_span:torch.Tensor, model:nn.Module):
        super().__init__()
        self.model, self.t_span = model, t_span
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch      
        t_eval, y_hat = self.model(x, t_span)
        y_hat = y_hat[-1] # select last point of solution trajectory
        loss = nn.CrossEntropyLoss()(y_hat, y)
        return {'loss': loss}   
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.01)

    def train_dataloader(self):
        return trainloader

In [42]:
from torchdyn.core import NeuralSDE

In [43]:
import torchsde

t_span = torch.linspace(0, 0.1, 100).to(device)
size = X.shape

bm = torchsde.BrownianInterval(
    t0=t_span[0],
    t1=t_span[-1],
    size=size,
    device=device,
    levy_area_approximation='space-time'
)

In [44]:
f = nn.Sequential(nn.Linear(2, 64), nn.Tanh(), nn.Linear(64, 2))
g = nn.Sequential(nn.Linear(2, 64), nn.Tanh(), nn.Linear(64, 2))


model = NeuralSDE(f, 
                  g,
                  solver='euler',
                  noise_type='diagonal',
                  sde_type='ito',
                  sensitivity='autograd',
                  t_span=t_span,
                  bm=bm
                 ).to(device)

In [45]:
learn = Learner(t_span, model)
if dry_run: trainer = pl.Trainer(min_epochs=1, max_epochs=1,gpus=1)
else: trainer = pl.Trainer(min_epochs=200, max_epochs=300,gpus=1)
trainer.fit(learn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | NeuralSDE | 644   
------------------------------------
644       Trainable params
0         Non-trainable params
644       Total params
0.003     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]